In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

%load_ext autoreload
%autoreload 2


In [2]:
from trust_eval.config import EvaluationConfig, ResponseGeneratorConfig
from trust_eval.evaluator import Evaluator
from trust_eval.logging_config import logger
from trust_eval.response_generator import ResponseGenerator
from trust_eval.retrieval import retrieve
from trust_eval.data import construct_data

# Get configs
generator_config = ResponseGeneratorConfig.from_yaml(yaml_path="generator_config.yaml")
logger.info(generator_config)

# Construct data
questions = ["Where do Asian elephants live?", 
         "Can Orthodox Jewish people eat shellfish?", 
         "How many species of sea urchin are there?", 
         "How heavy is an adult golden jackal?",
         "Who owns FX network?",
         "How many letters are used in the French alphabet?"]
answers = ["Asian elephants live in the forests and grasslands of South and Southeast Asia, including India, Sri Lanka, Thailand, and Indonesia.",
          "No, Orthodox Jewish people do not eat shellfish because it is not kosher according to Jewish dietary laws.",
          "There are about 950 species of sea urchins worldwide.",
          "An adult golden jackal typically weighs between 6 and 14 kilograms (13 to 31 pounds).",
          "FX network is owned by The Walt Disney Company.",
          "The French alphabet uses 26 letters, the same as the English alphabet."]

raw_docs = retrieve(questions, top_k=5)
data = construct_data(questions, answers, raw_docs, generator_config)

/home/shanghong/miniconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 2025-01-24 03:27:51,862 - INFO - trust_eval version 0.1.0 by Shang Hong Sim initialized.
 2025-01-24 03:27:52,111 - INFO - ResponseGeneratorConfig(yaml_path='generator_config.yaml', model='Qwen/Qwen2.5-0.5B-Instruct', data_type='asqa', prompt_file='prompts/asqa_rejection.json', eval_file='eval_data/custom_data.json', output_path='output/asqa_eval_top100_calibrated.json', quick_test=None, ndoc=5, shot=2, seed=42, no_doc_in_demo=False, fewer_doc_in_demo=False, ndoc_in_demo=None, no_demo=False, rejection=True, openai_api=False, azure=False, lora_path=None, vllm=True, temperature=0.5, top_p=0.95, max_new_tokens=300, max_length=8192, num_samples=1, rejection_flag="I apologize, but I couldn't find an answer", rejection_thres

Loading docs from pickle file...


 2025-01-24 03:28:15,061 - INFO - gtr embeddings found, loading...
 2025-01-24 03:28:28,580 - INFO - Running GTR retrieval...
 2025-01-24 03:28:33,603 - INFO - Loading AutoAIS model...
Loading checkpoint shards: 100%|██████████| 5/5 [00:11<00:00,  2.20s/it]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
 2025-01-24 03:28:46,712 - INFO - AutoAIS model loaded successfully.
 2025-01-24 03:28:51,191 - INFO - Data saved to eval_data/custom_data.json


In [4]:
generator_config = ResponseGeneratorConfig.from_yaml(yaml_path="generator_config.yaml")
logger.info(generator_config)

 2025-01-24 03:29:57,843 - INFO - ResponseGeneratorConfig(yaml_path='generator_config.yaml', model='Qwen/Qwen2.5-0.5B-Instruct', data_type='asqa', prompt_file='prompts/asqa_rejection.json', eval_file='eval_data/custom_data.json', output_path='output/asqa_eval_top100_calibrated.json', quick_test=None, ndoc=5, shot=2, seed=42, no_doc_in_demo=False, fewer_doc_in_demo=False, ndoc_in_demo=None, no_demo=False, rejection=True, openai_api=False, azure=False, lora_path=None, vllm=False, temperature=0.5, top_p=0.95, max_new_tokens=300, max_length=8192, num_samples=1, rejection_flag="I apologize, but I couldn't find an answer", rejection_threshold=85, autoais_model='google/t5_xxl_true_nli_mixture')


In [5]:
# Generator
generator = ResponseGenerator(generator_config)
data = generator.generate_responses(data)
generator.save_responses(output_path = "output/custom_data.json")

# Evaluate
evaluation_config = EvaluationConfig.from_yaml(yaml_path="eval_config.yaml")
logger.info(evaluation_config)
evaluator = Evaluator(evaluation_config)
evaluator.compute_metrics()
evaluator.save_results(output_path = "results/custom_data.json")


 2025-01-24 03:29:58,376 - INFO - Loading Qwen/Qwen2.5-0.5B-Instruct in torch.float16...


ValueError: `size` -2GB is not in a valid format. Use an integer for bytes, or a string with an unit (like '5.0GB').

In [ ]:
import datasets
hagrid = datasets.load_dataset("miracl/hagrid", split="dev")
print(hagrid[0])